In [36]:
import pandas as pd
import re
from tqdm import tqdm
import math
import numpy as np
import json
from ast import literal_eval

In [37]:
df_address = pd.read_excel("address.xlsx", sheet_name = 'ADDRESSES_Song_1非羈縻州縣')
df_address2 = pd.read_excel("address.xlsx", sheet_name = 'ADDRESSES_Tang_Raw')
df_ns=pd.read_csv('../data_dict/cbdb_entity_addresses.csv', sep='\t', header=None).rename(columns={0:'name', 1:'pos'})

In [38]:
# All QSW records.
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑')[['content_id', 'content', 'subject', 'author']].set_index('content_id')
df_qsw_raw['content']=[str(s) for s in df_qsw_raw['content']]
df_qsw_raw.dropna(inplace=True)
df_qsw_raw = df_qsw_raw[df_qsw_raw['content'] != "nan"]

/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [39]:
# Function for seperating sentences.
def sep_mark_sent(string):
    string=string.replace('，', '/w/').replace('；', '/w/').replace('、', '/w/')
    string=string.replace('！', '/w/').replace('。', '/w/').replace('？', '/w/').replace('！', '/w/')
    string=string.replace('：', '/w/')
    return [s for s in string.split("/w/") if s!='']

In [40]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    content=df_qsw_raw.loc[index]['content']
    sent_list=sep_mark_sent(content) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    kin_sent_list=[]
    content_id=index
    for sent in sent_list:
        if "進士" in sent:
            kin_sent_list.append(sent)
    df=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent, content] for sent in kin_sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent', 'content']
                                          ), df], axis=0, ignore_index=True
                            )

100%|██████████| 4701/4701 [00:05<00:00, 868.80it/s]


In [41]:
nianhao = pd.read_excel("Nianhao_SongRelated.xlsx")
nianhao

,c_nianhao_id,c_dy,c_dynasty_chn,c_nianhao_chn,c_nianhao_pin,c_firstyear,c_lastyear,Priority
0,387,6,唐,永貞,NaN,805,805,1
1,388,6,唐,元和,NaN,806,820,1
2,389,6,唐,長慶,NaN,821,824,1
3,390,6,唐,寶曆,NaN,825,827,1
4,391,6,唐,大和,NaN,827,835,1
...,...,...,...,...,...,...,...,...
228,634,18,元,元統,NaN,1333,1334,1
229,635,18,元,至元,NaN,1335,1340,1
230,636,18,元,至正,NaN,1341,1370,1
231,637,79,北元,宣光,NaN,1371,1378,1


In [42]:
tiangan = ['甲子','乙丑','丙寅','丁卯','戊辰','己巳','庚午','辛未','壬申','癸酉','甲戌','乙亥','丙子','丁丑','戊寅','己卯','庚辰','辛巳','壬午','癸未','甲申','乙酉','丙戌','丁亥',
'戊子','己丑','庚寅','辛卯','壬辰','癸巳','甲午','乙未','丙申','丁酉','戊戌','己亥','庚子','辛丑','壬寅','癸卯','甲辰','乙巳','丙午','丁未','戊申','己酉','庚戌','辛亥',
'壬子','癸丑','甲寅','乙卯','丙辰','丁巳','戊午','己未','庚申','辛酉','壬戌','癸亥']
string = ""
for t in tiangan:
    string = string + t + "|"
string = string[:-1]
string = "(年|" + string + ")"
string

'(年|甲子|乙丑|丙寅|丁卯|戊辰|己巳|庚午|辛未|壬申|癸酉|甲戌|乙亥|丙子|丁丑|戊寅|己卯|庚辰|辛巳|壬午|癸未|甲申|乙酉|丙戌|丁亥|戊子|己丑|庚寅|辛卯|壬辰|癸巳|甲午|乙未|丙申|丁酉|戊戌|己亥|庚子|辛丑|壬寅|癸卯|甲辰|乙巳|丙午|丁未|戊申|己酉|庚戌|辛亥|壬子|癸丑|甲寅|乙卯|丙辰|丁巳|戊午|己未|庚申|辛酉|壬戌|癸亥)'

In [43]:
for index in tqdm(df.index):
    row = df.loc[index]
    for i in nianhao.index:
        hao = nianhao.loc[i]
        if hao['c_nianhao_chn'] in row['sent']:
            re_rule = "(" + hao['c_nianhao_chn'] + "[^\/.]*?" + string + ")"
            df.loc[index, 'nianhao'] = hao['c_nianhao_chn']
            df.loc[index, 'time'] = hao['c_firstyear']
            l = re.findall(re_rule, row['sent'])
            time_list = []
            if l:
                for time in l:
                    time_list.append(time[0])
            else:
                time_list.append(hao['c_nianhao_chn'])
                
            df.loc[index, 'nianhaofull'] = str(time_list)

100%|██████████| 4595/4595 [01:31<00:00, 50.42it/s]


In [45]:
df = df.dropna()
df.sample(10)

,content_id,subject,author,sent,content,nianhao,time,nianhaofull
157,5720836.0,王自中,陳傅良,道甫登淳熙五年進士第,嗚呼!此吾友王君道甫之墓也。道甫諱自中，皇曾祖高，皇祖成子。父廷佐，贈朝奉郎。母施氏，封太安...,淳熙,1174.0,['淳熙五年']
1374,5827611.0,張啓女、黃厚之妻,戴栩,季適進士孔正大,初，余簿定海，定海宿樓船軍四千，與許浦相應援。主將吴君英開冶鑄，治札革，揉角煉幹，凡矛劍刀戟...,正大,1224.0,['正大']
1287,5834470.0,蹇君章,魏了翁,舉嘉定十年進士,潼之通泉蹇氏系出潯陽，伏羲時曰脩，秦穆時曰叔，世久莫可考。唐禧宗時曰元盛，扈駕爲排頓使，後居...,嘉定,1208.0,['嘉定十年']
689,5140780.0,狄棐,王安石,中咸平三年進士甲科,狄氏故并人，唐武后時，有以諒直至宰相者，有功中宗以及社稷，是爲梁公。公，梁公之十四世孫也。諱...,咸平,998.0,['咸平三年']
4385,5043760.0,何曮子,尹洙,周顯德二年進士第二人〔一〕,君諱某，字某，其先京兆咸陽人。祖諱朗，左司禦、率府率，葬河南新安，遂爲河南人。考諱曮，周顯德...,顯德,954.0,['顯德二年']
194,5888746.0,丁伯桂,劉克莊,公擢嘉泰三年進士第,寶、紹間一相擅國，所拔之士非鄞則婺，其言曰閩人難保。尤惡莆士，如陳宓、鄭寅之流，皆掃影滅迹，...,嘉泰,1201.0,['嘉泰三年']
3,5731931.0,劉千祥,雷孝友,登宣和甲辰進士,知沅州軍州、節制思南等處沿邊溪洞諸軍事劉恭叔奉其父軍州公行狀，詣余泣拜請銘。按狀，劉公諱千祥...,宣和,1119.0,['宣和甲辰']
1322,5834346.0,吳之巽,魏了翁,嘉定十年進士,人生而莫不有仁義之性具乎其心。禮經三百，威儀三千，聖人所以合内外之道而節文乎仁義者也。昔之教...,嘉定,1208.0,['嘉定十年']
4164,5093117.0,楊暈,蔡襄,景祐元年登進士第,予友楊公明，景祐元年登進士第，籍名下吏部銓，將補官，輒病，其年某月日終於京師。其友張平叔用浮...,景祐,1034.0,['景祐元年']
312,5713448.0,陳希點,樓鑰,擢紹聖四年進士第,公諱希點，字子與，處州青田人。陳姓出于有嬀，其來遠矣。九世祖名師訥，吴越王時爲銀青光禄大夫，...,紹聖,1094.0,['紹聖四年']


In [46]:
df.to_excel("jinshi.xlsx")

In [32]:
re_rule = "(一|元|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十)年"
digit = {'一': 1, '元': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十':10, '十一':11, '十二':12, '十三':13, '十四':14, '十五' :15, '十六':16, '十七' :17, '十八':18, '十九':19, '二十':20, '二十一':21, '二十二': 22, "二十三":23, "二十四":24, "二十五":25, "二十六" :26, "二十七" :27, "二十八":28,"二十九":29, '三十':30}
for index in tqdm(df.index):
    row = df.loc[index]
    find = re.findall(re_rule, row['sent'])
    if find:
        df.loc[index, 'year'] = find[0]
        #df.loc[index, 'time'] = digit.get(find[0]) + row['time']

100%|██████████| 660/660 [00:00<00:00, 4790.01it/s]


In [33]:
tiangan = ['甲子','乙丑','丙寅','丁卯','戊辰','己巳','庚午','辛未','壬申','癸酉','甲戌','乙亥','丙子','丁丑','戊寅','己卯','庚辰','辛巳','壬午','癸未','甲申','乙酉','丙戌','丁亥',
'戊子','己丑','庚寅','辛卯','壬辰','癸巳','甲午','乙未','丙申','丁酉','戊戌','己亥','庚子','辛丑','壬寅','癸卯','甲辰','乙巳','丙午','丁未','戊申','己酉','庚戌','辛亥',
'壬子','癸丑','甲寅','乙卯','丙辰','丁巳','戊午','己未','庚申','辛酉','壬戌','癸亥']

In [41]:
for index in tqdm(df.index):
    row = df.loc[index]
    for i in tiangan:
        if i in row['sent']:
            df.loc[index, 'tian'] = i
    if "甲科" in row['sent']:
        df.loc[index, 'rank'] = "甲科"
    if "乙科" in row['sent']:
        df.loc[index, 'rank'] = "乙科"


100%|██████████| 660/660 [00:00<00:00, 3582.35it/s]


In [50]:
df.sample(10)

,content_id,subject,author,sent,content,nianhao,time,year,tian,rank
4385,5043760.0,何曮子,尹洙,周顯德二年進士第二人〔一〕,君諱某，字某，其先京兆咸陽人。祖諱朗，左司禦、率府率，葬河南新安，遂爲河南人。考諱曮，周顯德...,顯德,958.0,二,NaN,NaN
1515,5787060.0,謝深甫,張嗣古,第乾道二年進士,宋中興四葉甲子夏六月辛酉，少傅致仕丞相謝公薨于私第。明年秋，葬臨海縣太平鄉環翠山之原。後二十...,乾道,1073.0,二,NaN,NaN
1592,5765130.0,王柟,葉適,以《春秋》中乾道丙戌進士第,公諱柟，字木叔，故順州王氏。石敬瑭叛，賄其地於狄，遷永嘉亭山，爲温州人。曾祖震，祖延齡，父贈...,乾道,1069.0,NaN,丙戌,NaN
3654,5163171.0,陳襄,孫覺,慶曆二年進士及第,公姓陳氏，諱襄，字述古。其先光州固始人，五代時王氏入閩，因隨家焉，今爲福州侯官縣古靈人。曾祖...,慶曆,1045.0,二,NaN,NaN
2362,5581060.0,陳璹,汪應辰,中建炎二年進士第,公陳氏，諱璹，字國壽，建之建陽人。曾祖譓，贈奉直大夫。祖某，奉議郎致仕。考賓，贈右朝請郎；妣...,建炎,1131.0,二,NaN,NaN
3451,5228442.0,王東美妻,蘇轍,肄及元祐九年進士第,伯父大中大夫生女子四人，仲姊適進士王君東美器之，獨享上壽，年七十有五。從其子肄爲梓州桐山尉，...,元祐,1104.0,九,NaN,NaN
623,5260777.0,黄夷仲,黄庭堅,登嘉祐六年進士第,黄氏本婺州金華人。公高祖諱贍〔二〕，當李氏時，來游江南，以策干中主，不能用，授著作佐郎，知分...,嘉祐,1068.0,六,NaN,NaN
3104,5309975.0,鄒夔,楊時,嘉祐中登進士第,先生諱某，字堯叟，姓鄒氏。其先出于魯國之邾。唐季之亂，避地閩中，故今爲邵武軍泰寧人。曾祖某，...,嘉祐,1056.0,NaN,NaN,NaN
125,5866089.0,鄭氏,方大琮,世系載《紹興三十年進士小録》云,王母夫人鄭，故虞部郎中倩之曾孫女，貢士彦輔之女。門族高華，三女皆賢淑，相攸卜妻，悉嬪于方。夫...,紹興,1210.0,三十,NaN,NaN
4006,5122152.0,蔡充,曾鞏,天聖二年進士及第,公諱充，字公度，天聖二年進士及第，爲邵武軍之邵武尉，又爲應天府之下邑尉。丁母夫人河間縣太君周...,天聖,1027.0,二,NaN,NaN


AttributeError: 'Series' object has no attribute 'isnan'